<div style="
    padding: 20px 20px;
    border-bottom: 5px solid rgba(121, 133, 148, 0.05);
">
    <img src="../shared/sf.svg" alt="Header" style="
        max-width: 100%; 
        height: auto; 
        max-height: 60px; 
        border-radius: 10px;
    ">
</div>


<img src="../shared/notebook-header.png" alt="Header" style="
    max-width: 100%; 
    height: auto; 
    max-height: 275px; 
    border-radius: 15px; /* Adjust the value to control the roundness */
">

# Service Cloud CSS Example

In this notebook, we provide an example similar to how we score Service Cloud usage in the real Customer Success Scorecard (CSS). We use fake data that was generated from theoretical distributions so this is cleaner than you would typically see in the wild, but serves our purpose of illutrating how Data Cloud enables our data science teams! The goal of this example is to walk the reader through setting up the AWS side of our architecture, such that it is ready to connect to Data Cloud. 

#### Table of Contents

1. **[Deploy AWS Infrastructure](#Deploy-AWS-Infrastructure)**
2. **[(Optional) Model Training](#(Optional)-Model-Training)**
3. **[Build and Push Docker Container](#Build-and-Push-Docker-Container)**
4. **[Deploy Endpoint](#Deploy-Endpoint)**
5. **[Data Cloud](#Data-Cloud)**
6. **[Clean Up](#Clean-Up)**

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## Deploy AWS Infrastructure

<div style="
    border: 2px solid #6fa8dc;
    background-color: rgba(121, 133, 148, 0.05); 
    padding: 1em; 
    margin-bottom: 1em; 
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <h3 style="margin-top: 0; font-size: 1.25em;"><b>Prerequisites</b></h3>
Before getting started we must have an AWS account where we have admin access. We recommend a new clean account that you can play with and see the patterns then recreate them in a way that makes sense in your production environment.
    <ul style="list-style-type: none; padding: 2; margin: 0;">
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
             <span>➡️ <a href="https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html" target="_blank" style="text-decoration: none; color: #0073e6;">Configure AWS CLI</a></span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
             <span>➡️ <a href="https://developer.hashicorp.com/terraform/tutorials/aws-get-started/install-cli" target="_blank" style="text-decoration: none; color: #0073e6;">Install Terraform</a></span>
        </li>
    </ul>
</div>

Packaged within the CSS repo is infrastructure code that deploys and configures a series of services within AWS allowing us to host and expose our model to Data Cloud. In short, we need an API proxy that wraps the underlying API exposed by SageMaker. See the architecture diagram below. There are three key components:

<ul style="list-style-type: none; padding: 0; margin-left: 2em;">
    <li style="display: flex; align-items: center; margin-bottom: 0.5em;">
        <img src="resources/iam.png" alt="Icon 1" style="width: 20px; height: 20px; margin-right: 0.5em;">
        <span><b>IAM — </b> IAM roles that give permissions to our services: (1) API Gateway can invoke the SageMaker endpoint and store CloudWatch logs and (2) SageMaker can perform necessary functions to deploy endpoint.</span>
    </li>
    <li style="display: flex; align-items: center; margin-bottom: 0.5em;">
        <img src="resources/api.png" alt="Icon 2" style="width: 20px; height: 20px; margin-right: 0.5em;">
        <span><b>API Gateway — </b> API Gateway wraps the named endpoint and logs all calls to Cloudwatch.</span>
    </li>
    <li style="display: flex; align-items: center;">
        <img src="resources/ecr.png" alt="Icon 3" style="width: 20px; height: 20px; margin-right: 0.5em;">
        <span><b>ECR — </b> We have an ECR repo where we will store our model container.</span>
    </li>
</ul>

<br>

<div style="text-align: center;">
    <img src="resources/aws-arch.png" alt="AWS Architecture" style="max-width: 100%; height: auto; max-height: 300px;">
</div>

#### Initialize Terraform Environment
First, we need to initialize our terraform environment.  This command only needs to be run once and will fetch the required AWS plugins.

In [ ]:
%%bash
cd ../../infrastructure
terraform init

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        color: #279425; 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Terraform has been successfully initialized!</pre>
</div>


#### Deploy Infrastructure

Below we call `apply` which deploys the infrastructure directly to AWS. The first block of output tells you the plan, and the second streamed output is its status.

In [ ]:
%%bash
cd ../../infrastructure
terraform apply -auto-approve

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        color: #279425; 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Apply complete! ...</pre>
</div>

#### Read Terraform Outputs

Lastly, we gather outputs from Terraform that we will plug into variables later in this notebook. Keep these private as we are dumping out secret keys which can be used to access your AWS account! We print a couple of these as you will need to copy and paste them into Data Cloud and Einstein Studio when we switch to the Salesforce portion of this tutorial.

In [ ]:
import json

terraform_output = !(cd ../../infrastructure && terraform output -json)
terraform_output = json.loads("".join(terraform_output))

match terraform_output:
    case {
        "api_url": {"value": api_url},
        "ecr_url": {"value": ecr_url},
        "iam_user": {"value": iam_user},
        "api_token": {"value": api_token},
        "model_name": {"value": model_name},
        "s3_bucket": {"value": s3_bucket},
        "sagemaker_execution_name": {"value": sagemaker_execution_name},
        "iam_user_secret": {"value": iam_user_secret},
        "iam_user_access_key": {"value": iam_user_access_key},
    }:
        pass

LJUST = 45
print(
    f"""Following are values that you can copy and paste into Data Cloud:
  - {"API Gateway URL".ljust(LJUST)}= \x1b[1m{api_url}\x1b[0m
  - {"API Gateway Token".ljust(LJUST)}= \x1b[1m{api_token}\x1b[0m
    {"".ljust(LJUST+2)}\x1b[31m⬆ DO NOT SHARE!\x1b[0m
  - {"Einstein Studio AWS User Access Key".ljust(LJUST)}= \x1b[1m{iam_user_access_key}\x1b[0m
  - {"Einstein Studio AWS User Secret".ljust(LJUST)}= \x1b[1m{iam_user_secret}\x1b[0m  
    {"".ljust(LJUST+2)}\x1b[31m⬆ DO NOT SHARE!\x1b[0m
  - {"S3 Bucket where we store data for Data Cloud".ljust(LJUST)}= \x1b[1m{s3_bucket}\x1b[0m
"""
)

#### Test Endpoint Connectivity

Even though we have not set up the underlying SageMaker endpoint, we can still test that it is operating correctly. We expect traffic to route to the non-existant endpoint and through a `NO_SUCH_ENDPOINT` error. It may take a few minutes for the API Gateway to acknowledge the API token.

In [ ]:
import json
import requests


def test_endpoint(api_url, api_token):
    response = requests.post(
        url=f"{api_url}/invocations",
        headers={
            "x-api-key": api_token,
            "Content-Type": "application/json",
        },
        json={},
    )
    json_response = response.json()
    user_out = json.dumps(json_response, indent=2)
    print(f"\x1b[37mRESPONSE={user_out}\x1b[0m")
    return response.json()


response_data = test_endpoint(api_url, api_token)

match response_data:
    case {"ErrorCode": "NO_SUCH_ENDPOINT"}:
        print("\x1b[1;32mAPI correctly configured!\x1b[0m")
    case {"ErrorCode": "CLIENT_ERROR_FROM_MODEL"}:
        print(
            "\x1b[1;31mThere is already a model hosted. Make sure to tear it down!\x1b[0m"
        )
    case _:
        print(
            "\x1b[1;31mSomething went wrong! "
            "Check in the console to make sure it is correctly configured!\x1b[0m"
        )

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        color: #279425; 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
API correctly configured!</pre>
</div>

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## (Optional) Model Training

In this section, we train the model from scratch. If you just want to walk through the pattern, you can skip this! We provide sample artifacts that you can use for deployment in the `artifacts/` folder.


<div style="
    background-color: rgba(111, 168, 220, 0.1); 
    border-color: rgba(111, 168, 220, 0.6); 
    border-left: 5px solid rgba(111, 168, 220, 1); 
    padding: 0.5em;
">
    <b>NOTE:</b> The focus here is on deployment, so we omit details on how we develop the configuration file, which can be thought of as our hyperparameters. For our custom model, these hyperparameters are found through intensive analyses by decision scientists.
</div>


<br>

<div style="text-align: center;">
    <img src="resources/model-training.png" alt="AWS Architecture" style="max-width: 80%; height: auto; max-height: 300px;">
</div>

#### Install CSS
The code in this section depends on the Python package defined in this repository. While you can install this remotely, it makes sense to install <b>this</b> version of the code to make sure it is synced. 

In [ ]:
import sys
import importlib
import os

! cd ../.. && pip install -e ".[deploy]"

try:
    import css
except ImportError as exc:
    print("Manually adding path to avoid notebook restart...")
    package_path = os.path.abspath(os.path.join("..", ".."))
    sys.path.append(package_path)
    importlib.invalidate_caches()
    import css

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Successfully installed css-...</pre>
</div>

#### Set Up Logging

We want to stream the model fit logs to standard output so we can observe the model being fit. 

In [ ]:
import logging
import sys

logging.basicConfig(
    format="%(asctime)s %(levelname)s -- %(name)s: %(message)s",
    level=logging.INFO,
    stream=sys.stdout,
)

#### Pull in Training Data

We use data provided in the `data/` folder and merge it together into a single source. In production, this data will come from a mix of Data Cloud and CRM so we mimic that during training.

In [ ]:
import pandas as pd

all_data = pd.read_csv("data/usage_metrics.csv", index_col=0)
_product_data = pd.read_csv("data/product_metrics.csv", index_col=0)
_peer_dims_data = pd.read_csv("data/peer_dims.csv", index_col=0)

all_data[_product_data.columns] = _product_data
all_data[_peer_dims_data.columns] = _peer_dims_data

#### Deserialize the Configuration File into a Model Object

We define model structures as configuration YAML files. Our `ConfigModel` reads this into a our structure then converts it to a model object (`service_model`) which is untrained but preconfigured with our hyperparameters.

In [ ]:
from css import config

service_model_config = config.ConfigModel.from_yaml("config.yaml")
service_model = service_model_config.to_obj()

#### Train the Model On Our Data

We fit the model. It iterates metric by metric. You can ignore the warnings! These are fitting logs that can be tracked later, we are technically fitting thousands of distributions in this step so we expect a lot of noise. Our $R^2$ values are all 1 because we are using nearest neighbors and theoretical distributions.

In [ ]:
service_model.fit(all_data)

#### Serialize Model for Deployment

Finally, we serialize the Python object as a pickle file then wrap the pickle file in a tarfile. This is how SageMaker expects model data. 

In [ ]:
import pickle as pkl
import tarfile

# Serialize model
with open("artifacts/css-model", "wb") as f:
    pkl.dump(service_model, f)

# Compress model file for SageMaker
tar_filename = "artifacts/model_data.tar.gz"
with tarfile.open(tar_filename, "w:gz") as tar:
    tar.add("artifacts/css-model", arcname="css-model")

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## Build and Push Docker Container

<div style="
    border: 2px solid #6fa8dc; 
    background-color: rgba(121, 133, 148, 0.05); 
    padding: 1em; 
    margin-bottom: 1em; 
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <h3 style="margin-top: 0; font-size: 1.25em;"><b>Prerequisites</b></h3>
    Docker must be installed and the daemon must be running.
    <ul style="list-style-type: none; padding: 2; margin: 0;">
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
            <span>➡️ 
            <a href="https://docs.docker.com/engine/install/" target="_blank" style="text-decoration: none; color: #0073e6;">
                Install Docker
            </a></span>
        </li>
    </ul>
</div>

<p>
    We need a container that serves as the environment where our model will run. Below, we will build the container defined in our <code>Dockerfile</code> in the package.
</p>


### Get Constants

In the following cell, we will gather the information we need to deploy our model to ECR.

In [ ]:
css_version = css.__version__.replace("+", "_")
full_tag_name = f"{ecr_url}:{css_version}"
region = full_tag_name.split(".")[3]

#### Login to ECR from Local

We need to get a docker login such that we can push our image to ECR from local. If run correctly, this should print `Login Succeeded`.

In [ ]:
command = (
    f"aws ecr get-login-password --region {region} "
    f"| docker login --username AWS --password-stdin {full_tag_name}"
)
!{command}

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Login Succeeded</pre>
</div>

### Build the Container

We will build the container locally then tag with the correct ECR name. This may take a while, especially the first time!

In [ ]:
!DOCKER_CLI_HINTS=false docker build --progress=plain -t css_mini ../..
!docker tag css_mini {full_tag_name}

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
#14 DONE X.Xs</pre>
</div>

### Push the Container to ECR

We can now push the container to the remote repository. If this step doesn't work, your ECR repository was not correctly configured.

In [ ]:
!docker push {full_tag_name}

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
...: Pushed     XXXMB/XXXMB</pre>
</div>

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## Deploy Endpoint

<div style="
    border: 2px solid #6fa8dc; 
    background-color: rgba(121, 133, 148, 0.05); 
    padding: 1em; 
    margin-bottom: 1em; 
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <h3 style="margin-top: 0; font-size: 1.25em;"><b>Prerequisites</b></h3>
    We need everything to have been run correctly up to this point — excluding model training, unless specifically desired.
    <ul style="list-style-type: none; padding: 2; margin: 0;">
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
            ➡️ AWS infra deployed
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
            ➡️ Docker container built and pushed to ECR
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
            <span>➡️ <code>sagemaker</code> Python package installed</span>
        </li>
    </ul>
</div>


#### Deploy to SageMaker

In the following cell, we create a function that we will use to host our model as an endpoint and execute with values generated from previous cells. 

SageMaker prints a dash (`-`) per minute to indicate loading. This may take up to five minutes. Once it is successfully deployed, it shows an exclamation mark (`!`).

In [ ]:
import sagemaker
import boto3


def deploy(
    fit_model_dump: str,
    ecr_image: str,
    execution_role: str,
    instance_type: str = "ml.t2.medium",
) -> sagemaker.predictor.Predictor:
    """Deploy CSS to SageMaker remote.

    We upload our model data to S3, then use it to create a model definition in
    SageMaker. This model is then deployed to an endpoint instance called
    ``css-mini-service-endpoint``. We return a ``Predictor`` that can be used
    to test that the model is responding as expected.

    Args:
        fit_model_dump: String path to our compressed model data.
        ecr_image: String name of specific ECR image.
        execution_role: Defaults to our Terraform IAM role.
        instance_type: Defaults to a small machine which should be all we need
            for this demo!
    """
    sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
    model_data = sagemaker_session.upload_data(
        fit_model_dump, key_prefix="css/model_data"
    )

    model = sagemaker.model.Model(
        ecr_image,
        role=execution_role,
        sagemaker_session=sagemaker_session,
        model_data=model_data,
        predictor_cls=sagemaker.predictor.Predictor,
        env={"MODEL_DATA": model_data},
    )

    predictor = model.deploy(
        instance_type=instance_type,
        initial_instance_count=1,
        endpoint_name=model_name,
    )
    predictor.serializer = sagemaker.serializers.JSONSerializer()
    predictor.deserializer = sagemaker.deserializers.JSONDeserializer()
    return predictor


predictor = deploy(
    fit_model_dump="artifacts/model_data.tar.gz",
    ecr_image=full_tag_name,
    execution_role=sagemaker_execution_name,
)

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    SageMaker prints a dash per minute to indicate loading. This may take up to five minutes. Once it is successfully deployed, it shows an exclamation mark.
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
-----!</pre>
</div>


#### Test the Remote Endpoint
We will send some null data to the endpoint and make sure that it is sending back predictions as expected.

In [ ]:
all_zeroes = {"instances": [{"features": [0] * 15}]}
predictions = predictor.predict(all_zeroes)

print(f"\x1b[37mRESPONSE={json.dumps(predictions, indent=2)}\x1b[0m")
match predictions:
    case {"predictions": [{"last_modified": _}]}:
        print("\x1b[1;32mGot expected response!\x1b[0m")
    case _:
        print("\x1b[1;31mSomething went wrong! Check Cloudwatch logs to debug.\x1b[0m")

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Got expected response!</pre>
</div>

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## Data Cloud

<div style="
    border: 2px solid #6fa8dc; 
    background-color: rgba(121, 133, 148, 0.05); 
    padding: 1em; 
    margin-bottom: 1em; 
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <h3 style="margin-top: 0; font-size: 1.25em;"><b>Prerequisites</b></h3>
    We will use Salesforce Data Cloud to connect to our endpoint above:
    <ul style="list-style-type: none; padding: 2; margin: 0;">
        <li style="display: flex; align-items: center; margin-bottom: 0.5em; margin-top: 0.5em;">
            ➡️ Data Cloud instance that has been set up
        </li>
    </ul>
</div>


### Upload data
We will load some data to our S3 bucket that Data Cloud will register as a Data Stream and track it for changes in data.

In [ ]:
import boto3

s3_client = boto3.Session().client("s3")
s3_client.upload_file(
    "data/product_metrics.csv", s3_bucket, "product_metrics/product_metrics.csv"
)
s3_client.upload_file(
    "data/usage_metrics.csv", s3_bucket, "usage_metrics/usage_metrics.csv"
)

<table style="width: 65%; min-width: 800px; margin: 40px auto; table-layout: fixed;">
  <tr style="background-color: rgba(255,255,255,0) !important;">
    <!-- Content Box -->
    <td style="vertical-align: top; width: 50%; padding-right: 5px;">
      <div style="
          background-color: rgba(121, 133, 148, 0.05); 
          padding: 40px; 
          border-radius: 18px; 
          margin-left: auto;
          margin-right: 0;
          max-width: 75%;
          min-width: 200px;
      ">
        <h2>Stop here and switch over to Data Cloud tutorial!</h2>
        <p>With all of this done, we can now walk through the Data Cloud portion of the tutorial in <em>developers.salesforce.com</em>.</p>
        <div style="text-align: center;">
          <a href="https://www.example.com" target="_blank">
            <div style="
                background: linear-gradient(33deg, #359ddc, #6abdef);
                border: none;
                color: white;
                padding: 10px 20px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 16px;
                margin-top: 20px;
                cursor: pointer;
                border-radius: 8px;
                box-shadow: 2px 2px 6px rgba(0, 0, 0, 0.2), 2px 2px 4px rgba(121, 133, 148, 0.5);
            ">Go to Example</div>
          </a>
        </div>
      </div>
    </td>
    <!-- Image -->
    <td style="vertical-align: center; width: 20%; text-align: left; padding-left: 20px;">
      <img src="../shared/genie.webp" style="height: auto; max-height: 300px; max-width: 100%; min-width: 200px;">
    </td>
  </tr>
</table>


#### Test Data Cloud Streaming
Let's update some of our data in S3 and see it arrive in Data Cloud then trigger a prediction job! We have another CSV where we update the product metrics for a subset of users. We show the change below, then update the file in S3.

In [ ]:
import pandas as pd
import numpy as np

s3_client.upload_file(
    "data/product_metrics_update.csv",
    s3_bucket,
    "product_metrics/product_metrics_update.csv",
)


def show_change(old: pd.DataFrame, new: pd.DataFrame) -> pd.io.formats.style.Styler:
    columns = [c for c in old.columns if c != "last_updated"]
    frames = []
    for c in columns:
        new_series = new[c]
        col_df = old[c].to_frame().reindex(new_series.index)
        col_df["diff"] = np.where(new_series - col_df[c] > 0, "↗", "↘")
        col_df["new"] = new_series
        col_mi = pd.MultiIndex.from_product([[c], ["old", "", "new"]])
        col_df.columns = col_mi
        frames.append(col_df)
    analysis_df = pd.concat(frames, axis=1)

    def highlight_greater_less(s):
        if s.name[1] == "new":
            x = analysis_df[(s.name[0], "old")]
            return [
                "color: green" if s_i > x_i else "color: red" for s_i, x_i in zip(s, x)
            ]
        return ["color: lightgrey"] * len(s)

    last_updated_old = old.last_updated.iloc[0]
    last_updated_new = new.last_updated.iloc[0]
    caption = (
        "Data changed for accounts from "
        f"<b>{last_updated_old}</b> to <b>{last_updated_new}</b>"
    )
    caption_styles = {}
    caption_styles["selector"] = "caption"
    caption_styles["props"] = "caption-side: bottom; text-align: right;"
    return (
        analysis_df.style.apply(highlight_greater_less, axis=0)
        .set_caption(caption)
        .set_table_styles([caption_styles], overwrite=False)
    )


old_product_metrics = pd.read_csv("data/product_metrics.csv", index_col=0)
new_product_metrics = pd.read_csv("data/product_metrics_update.csv", index_col=0)
show_change(old_product_metrics, new_product_metrics)

<br><br>
<div style="border-top: 15px solid rgba(121, 133, 148, 0.05); border-radius: 15px; width: 75%; margin: 20px auto;"></div>
<br><br>

## Clean Up

Once you are done, you can tear down your endpoint with the following. This will make your model inaccessible, so don't do this until you have completed the entire tutorial!

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(model_name)
sagemaker.Session().delete_endpoint_config(model_name)

<div style="
    border: 1px solid rgba(121, 133, 148, 0.2); 
    border-radius: 5px; 
    padding: 10px; 
    background-color: rgba(121, 133, 148, 0.05); 
    margin-top: 10px; 
    max-width: 600px;
    box-shadow: 0 0 10px rgba(121, 133, 148, 0.05);
">
    <strong>Expected Output:</strong>
    <pre style="
        background-color: rgba(121, 133, 148, 0.1); 
        padding: 10px; 
        border-radius: 5px; 
        margin-top: 5px;
    ">
Deleting endpoint with name: ...
Deleting endpoint configuration with name: ...</pre>
</div>

You can also remove all of the infrastructure deployed to your AWS account with this command.

In [ ]:
%%bash
cd ../../infrastructure
terraform destroy -auto-approve

<br><br>
<div style="text-align: center;">
    <img src="../shared/divider.png" alt="AWS Architecture" style="max-width: 100%; height: auto; max-height: 50px;">
</div>